# Setup and installation

SuperDuperDB uses MongoDB for data storage. If you haven't done so already, install it using the following lines of bash.

```bash
wget -qO - https://www.mongodb.org/static/pgp/server-6.0.asc | sudo apt-key add -
sudo apt-get install gnupg
wget -qO - https://www.mongodb.org/static/pgp/server-6.0.asc | sudo apt-key add -
echo "deb [ arch=amd64,arm64 ] https://repo.mongodb.org/apt/ubuntu focal/mongodb-org/6.0 multiverse" | sudo tee /etc/apt/sources.list.d/mongodb-org-6.0.list
sudo apt-get update
sudo apt-get install -y mongodb-org
```

If you haven't downloaded the data already, execute the lines of bash below. We've tried to keep it clean,
and for reasons of efficiency have resized the images using imagemagick:

```bash
curl http://images.cocodataset.org/annotations/annotations_trainval2014.zip -o raw.zip
unzip raw.zip
curl http://images.cocodataset.org/zips/train2014.zip -o images.zip
unzip -q images.zip
sudo apt install -y imagemagick
mogrify -resize 224x 'train2014/*.jpg'
mkdir -p data/coco
mv train2014 data/coco/images
mv annotations/* data/coco
rm raw.zip
rm images.zip
```

In case you haven't done so already, install the dependencies for this tutorial, including SuperDuperDB,
which is a simple pip install:

```bash
pip install pandas
pip install pillow
pip install torch
pip install -r ../../requirements.txt
```

SuperDuperDB can handle data in any format, including images. The documents in the database are MongoDB `bson` documents, which mix `json` with raw bytes and `ObjectId` objects. SuperDuperDB takes advantage of this by 
serializing more sophisticated objects to bytes, and reinstantiating the objects in memory, when data is queried.


In order to tell SuperDuperDB what type an object has, one specifies this with a subdocument of the form:

```json
{
    "_content": {
        "bytes": ...,
        "type": "<my-type>",
    }
}
```

If however, the content is located on the web or the filesystem, one can specify the URLs directly:

```json
{
    "_content": {
        "url": "<url-or-file>",
        "type": "<my-type>",
    }
}
```

Let's see this now in action. We reformat the CoCo data, so that each image is associated in one document with all of the captions which describe it, and add the location of the images using the `_content` keyword.

In [1]:
import json

with open('data/coco/captions_train2014.json') as f:
    raw = json.load(f)
    
raw['images'] = {x['id']: x for x in raw['images']}

for im in raw['images']:
    raw['images'][im]['captions'] = []
    
for a in raw['annotations']:
    raw['images'][a['image_id']]['captions'].append(a['caption'])

raw = list(raw['images'].values())

for i, im in enumerate(raw):
    # if image is already in memory, then add 'bytes': b'...' instead of 'url': '...'
    # for content located on the web, use 'http://' or 'https://' instead of 'file://'
    im['img'] = {
        '_content': {'url': f'file://data/coco/images/{im["file_name"]}', 'type': 'image'}
    }
    raw[i] = {'captions': im['captions'], 'img': im['img']}
    
print(json.dumps(raw[:2], indent=2))

[
  {
    "captions": [
      "A restaurant has modern wooden tables and chairs.",
      "A long restaurant table with rattan rounded back chairs.",
      "a long table with a plant on top of it surrounded with wooden chairs ",
      "A long table with a flower arrangement in the middle for meetings",
      "A table is adorned with wooden chairs with blue accents."
    ],
    "img": {
      "_content": {
        "url": "file://data/coco/images/COCO_train2014_000000057870.jpg",
        "type": "image"
      }
    }
  },
  {
    "captions": [
      "A man preparing desserts in a kitchen covered in frosting.",
      "A chef is preparing and decorating many small pastries.",
      "A baker prepares various types of baked goods.",
      "a close up of a person grabbing a pastry in a container",
      "Close up of a hand touching various pastries."
    ],
    "img": {
      "_content": {
        "url": "file://data/coco/images/COCO_train2014_000000384029.jpg",
        "type": "image"
      }

You can see that we've slightly reformatted the data in preparation for injestion to SuperDuperDB. That's so that SuperDuperDB knows how to handle the content behind the URLs or file paths for the images. We'll learn more about that in the next chapter.

In [2]:
with open('data/coco/data.json', 'w') as f:
    json.dump(raw, f)